In [15]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import psycopg2

In [16]:
netflix_file = "netflix_titles_nov_2019.csv"
netflix_df = pd.read_csv(netflix_file)
netflix_df.head()

show_id                                title                    director  \
0  81193313                            Chocolate                         NaN   
1  81197050  Guatemala: Heart of the Mayan World  Luis Ara, Ignacio Jaunsolo   
2  81213894                      The Zoya Factor             Abhishek Sharma   
3  81082007                            Atlantics                   Mati Diop   
4  80213643                      Chip and Potato                         NaN   

                                                cast  \
0  Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...   
1                                  Christian Morales   
2  Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...   
3  Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...   
4  Abigail Oliver, Andrea Libman, Briana Buckmast...   

                    country         date_added  release_year rating  \
0               South Korea  November 30, 2019          2019  TV-14   
1                       NaN  November 30, 2019          2019   TV-G   
2                     India  November 30, 2019          2019  TV-14   
3  France, Senegal, Belgium  November 29, 2019          2019  TV-14   
4    Canada, United Kingdom                NaN          2019   TV-Y   

    duration                                          listed_in  \
0   1 Season  International TV Shows, Korean TV Shows, Roman...   
1     67 min                Documentaries, International Movies   
2    135 min             Comedies, Dramas, International Movies   
3    106 min   Dramas, Independent Movies, International Movies   
4  2 Seasons                                           Kids' TV   

                                         description     type  
0  Brought together by meaningful meals in the pa...  TV Show  
1  From Sierra de las Minas to Esquipulas, explor...    Movie  
2  A goofy copywriter unwittingly convinces the I...    Movie  
3  Arranged to marry a rich man, young Ada is cru...    Movie  
4  Lovable pug Chip starts kindergarten, makes ne...  TV Show

In [43]:
netflix_cols=["show_id","title","type"]
netflix_transformed=netflix_df[netflix_cols].copy()
netflix_transformed=netflix_transformed.rename(columns={"show_id":"id","type":"film_or_series"})
netflix_transformed.head()

id                                title film_or_series
0  81193313                            Chocolate        TV Show
1  81197050  Guatemala: Heart of the Mayan World          Movie
2  81213894                      The Zoya Factor          Movie
3  81082007                            Atlantics          Movie
4  80213643                      Chip and Potato        TV Show

In [24]:
golden_file = "golden_globe_awards.csv"
golden_df = pd.read_csv(golden_file)
golden_cols=["category","nominee","film"]
golden_transformed = golden_df[golden_cols].copy()
df=golden_transformed.fillna("")
df.head()

category                 nominee  \
0  Best Performance by an Actress in a Supporting...          Katina Paxinou   
1  Best Performance by an Actor in a Supporting R...           Akim Tamiroff   
2                     Best Director - Motion Picture              Henry King   
3                                            Picture  The Song Of Bernadette   
4                          Actress In A Leading Role          Jennifer Jones   

                      film  
0  For Whom The Bell Tolls  
1  For Whom The Bell Tolls  
2   The Song Of Bernadette  
3                           
4   The Song Of Bernadette

In [28]:
for j in range(len(df)): 
    if df['film'][j]=="":
       df['film'][j]=df['nominee'][j]
df.head()

category                 nominee  \
0  Best Performance by an Actress in a Supporting...          Katina Paxinou   
1  Best Performance by an Actor in a Supporting R...           Akim Tamiroff   
2                     Best Director - Motion Picture              Henry King   
3                                            Picture  The Song Of Bernadette   
4                          Actress In A Leading Role          Jennifer Jones   

                      film  
0  For Whom The Bell Tolls  
1  For Whom The Bell Tolls  
2   The Song Of Bernadette  
3   The Song Of Bernadette  
4   The Song Of Bernadette

In [40]:
type(df)
df.drop('nominee',axis=1,inplace=True)

In [41]:
golden_transformed_final=df.copy()
golden_transformed_final.head()

category                     film
0  Best Performance by an Actress in a Supporting...  For Whom The Bell Tolls
1  Best Performance by an Actor in a Supporting R...  For Whom The Bell Tolls
2                     Best Director - Motion Picture   The Song Of Bernadette
3                                            Picture   The Song Of Bernadette
4                          Actress In A Leading Role   The Song Of Bernadette

In [42]:
new_series=pd.Series([])
for i in range(len(golden_transformed_final)): 
    new_series[i]=i+1
    
golden_transformed_final.insert(0,"serial_no",new_series)
golden_transformed_final.head(100)

serial_no                                           category  \
0           1  Best Performance by an Actress in a Supporting...   
1           2  Best Performance by an Actor in a Supporting R...   
2           3                     Best Director - Motion Picture   
3           4                                            Picture   
4           5                          Actress In A Leading Role   
..        ...                                                ...   
95         96  Best Performance by an Actor in a Supporting R...   
96         97                     Best Director - Motion Picture   
97         98                     Best Director - Motion Picture   
98         99                     Best Director - Motion Picture   
99        100                     Best Director - Motion Picture   

                       film  
0   For Whom The Bell Tolls  
1   For Whom The Bell Tolls  
2    The Song Of Bernadette  
3    The Song Of Bernadette  
4    The Song Of Bernadette  
..                      ...  
95            All About Eve  
96         Sunset Boulevard  
97           Born Yesterday  
98       The Asphalt Jungle  
99            All About Eve  

[100 rows x 3 columns]

In [11]:
a = golden_df.category.unique()
a.sort()
sr = pd.Series(a)
gg_category=pd.DataFrame(a)
b=sr.str.contains('Best Motion Picture',regex=True)
c=pd.DataFrame(b)
indexnames = gg_category[c[0]==False].index
indexnames
gg_wanted=gg_category
gg_wanted.drop(indexnames,inplace=True)
gg_wanted.head()

0
18           Best Motion Picture - Animated
19              Best Motion Picture - Drama
20   Best Motion Picture - Foreign Language
21  Best Motion Picture - Musical or Comedy

In [52]:
rds_connection_string = "postgres:postgres@localhost:5432/Netflix_Golden_Final"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [54]:
netflix_transformed.to_sql(name='netflix_db', con=engine, if_exists='append', index=False)

In [55]:
golden_transformed_final.to_sql(name='golden_db', con=engine, if_exists='append', index=False)

In [56]:
pd.read_sql_query('select * from netflix_db', con=engine).head()

id                                title film_or_series
0  81193313                            Chocolate        TV Show
1  81197050  Guatemala: Heart of the Mayan World          Movie
2  81213894                      The Zoya Factor          Movie
3  81082007                            Atlantics          Movie
4  80213643                      Chip and Potato        TV Show

In [57]:
pd.read_sql_query('select * from golden_db', con=engine).head()

serial_no                                           category  \
0          1  Best Performance by an Actress in a Supporting...   
1          2  Best Performance by an Actor in a Supporting R...   
2          3                     Best Director - Motion Picture   
3          4                                            Picture   
4          5                          Actress In A Leading Role   

                      film  
0  For Whom The Bell Tolls  
1  For Whom The Bell Tolls  
2   The Song Of Bernadette  
3   The Song Of Bernadette  
4   The Song Of Bernadette